In [70]:
from numpy import sum
from nibabel import load
from glob import glob
from pandas import Series, DataFrame
from datetime import date
from os import path
from nipype.interfaces import freesurfer as fs

In [71]:
home = '/Volumes/iang/active/BABIES/BABIES_ibeat/processed_brains/BABIES_LT2'
infiles = home + '/processed_T2w'
subs = Series(glob(infiles + '/*-skullstripped_anat.nii')).str.replace(infiles + '/',"").sort_values().str.rstrip('-skullstripped_anat.nii')
output = home +'/binarized_masks'

In [83]:
def count_voxels(data):
    num_vox = sum(data.get_data() > 0)
    return num_vox

In [93]:
for sub in subs:
    if path.exists(output + '/' + sub+'-skullstripped_anat_mask.nii'):
        print("Mask already created for {}".format(sub))
    else:
        binvol = fs.Binarize(in_file = infiles + '/' +sub +'-skullstripped_anat.nii',
                             min=0,
                             max = 1,
                             invert = True,
                             binary_file= output + '/' + sub+'-skullstripped_anat_mask.nii')
        binvol.run()

Mask already created for 1005
Mask already created for 1010
Mask already created for 1011
Mask already created for 1017
Mask already created for 1020
Mask already created for 1023
Mask already created for 1027
Mask already created for 1028
Mask already created for 1033
Mask already created for 1036
Mask already created for 1037
Mask already created for 1038
Mask already created for 1042
Mask already created for 1043
Mask already created for 1050
Mask already created for 1053
Mask already created for 1057
Mask already created for 1060
Mask already created for 1068
Mask already created for 1069
Mask already created for 1071
Mask already created for 1078


In [98]:
final = DataFrame(columns = ['ID', 'ICV (mm^3)'])

for sub in subs:
    data = load(output+'/' +sub+'-skullstripped_anat_mask.nii')
    vox_count = count_voxels(data)
    results = Series([sub, vox_count], index = ['ID', 'ICV (mm^3)'])
    final = final.append(results, ignore_index = True)  


In [99]:
today = str(date.today())
final.to_csv(home+'/Newborn_ICV_'+today+'.csv')